<a href="https://colab.research.google.com/github/aarjavid/DataMining/blob/main/AssociationRules_AprioriAlgo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time

def isSuperSetofNFST(intersection,disjoint) :

    if(len(intersection) == 0):
        return 0
    this_hashcode = getUniqueHashCode(disjoint)
    if this_hashcode in NFST_hash:
        return 1
    for elem in intersection:
        temp = []
        temp = [elem] + disjoint
        temp.sort()
        this_hashcode = getUniqueHashCode(temp)
        if this_hashcode in NFST_hash: 
            return 1
    return 0

def getIntersectingElements(list1,list2) :
    intersectList = []
    for elemOfList1 in list1:
        if elemOfList1 in list2:
            intersectList.append(elemOfList1)
    intersectList.sort()
    return intersectList
        

def getDisjointElements(list1,list2) :
    disjointList = []
    for elemOfList1 in list1:
        if elemOfList1 not in list2:
            disjointList.append(elemOfList1)
    for elemOfList2 in list2:
        if elemOfList2 not in list1:
            disjointList.append(elemOfList2)
    disjointList.sort()
    return disjointList
        

def getUniqueHashCode(list) :
    NewHashCode = 0
    iter1 = 0
    for i in list:
        this_single_item_code = HashCode[i]
        NewHashCode += this_single_item_code * pow(1000,iter1)
        iter1 += 1
    return NewHashCode
        
def getEventOccurence(event,alg=1):
    Occur = 0
    for this_trans in transactions:
        flag = 1
        for elem in event:
            if elem not in this_trans:
                flag = 0
        if flag:
            Occur += 1
            
    if(Occur > 0):
        event.sort()
        this_hashcode = getUniqueHashCode(event)      
        if(alg==1):
            OccurenceCount[this_hashcode] = Occur
        if(alg==2):
            OccurenceCountBF[this_hashcode] = Occur
    return Occur

def issubset(list1,list2):
    return set(list1).issubset(list2)

#Below function to generate FST by brute force alg for a set of K elements
def getFSTfromAllComb(SampleList):
    FSTbf = []
    for event in SampleList:
        eventCount = getEventOccurence(event,2)
        if(eventCount >= minoccur):
            FSTbf.append(event)
    return FSTbf

#Generate all possible combinations of length k
def GenCombin(lst, lst_tmp, start,end, index,k) :

    if (index == k) :
        lst1_tmp = lst_tmp[0:k]
        
        CombinationList.append(lst1_tmp)
        return 
          
    i = start
    while(i<=end and end-i+1 >= k-index):
        lst_tmp[index] = lst[i]
        GenCombin(lst, lst_tmp, i+1, end, index+1, k)
        i += 1
        

#Geneate FST by Apriori Alg        
def getFrequentItemSets_Apr(FSTk,newK,minoccur):
    
    N = len(FSTk)
    listOfListsTemp = []
    listOfLists = []
    intersection = []
    disjoint = []
    temp = []
    for elem1 in range (0,N-1):
        for elem2 in range (elem1+1, N ):
            intersection = getIntersectingElements(FSTk[elem1],FSTk[elem2])
            disjoint = getDisjointElements(FSTk[elem1],FSTk[elem2])
            temp = intersection + disjoint
            temp.sort()
            if(len(temp) == newK):
                if temp not in listOfLists:
                    this_hashcode = getUniqueHashCode(temp)
                    isNFST = isSuperSetofNFST(intersection,disjoint)
                    if isNFST == 1 : 
                        NFST_hash[this_hashcode] = 1
                    elif(getEventOccurence(temp)  < minoccur):
                        NFST_hash[this_hashcode] = 1
                    else:
                        listOfLists.append(temp)
    return listOfLists



def genAssrules(FST,alg=1):
    AllAssRules = []
    for thislist in FST:
        for smthislist in FST:
            if (issubset(smthislist,thislist)) :
                a1 = getIntersectingElements(smthislist,thislist)
                a2 = getDisjointElements(smthislist,thislist)
                if (len(a1)>0 and len(a2)>0):
                    a1.sort()
                    a2.sort()
                    comb = a1 + a2 
                    sortedcomb = sorted(comb)
                    if(alg==1):
                        Sup_list = OccurenceCount[getUniqueHashCode(sortedcomb)]
                        Sup_list1 = OccurenceCount[getUniqueHashCode(a1)]
   
                    if(alg==2):
                        Sup_list = OccurenceCountBF[getUniqueHashCode(sortedcomb)]
                        Sup_list1 = OccurenceCountBF[getUniqueHashCode(a1)]
  
                    sup = str(round((Sup_list/TotalTransact)*100,2)) + "%"
                    conf12 = Sup_list/Sup_list1
                    if(conf12 >= minConf):
                        lista = []
                        lista.append(a1)
                        lista.append(a2)
                        lista.append(sup)
                        conf = str(round(conf12*100,2)) + "%"
                        lista.append(conf)
                        if lista not in AllAssRules:
                            AllAssRules.append(lista)

    return AllAssRules
                            

#Program Starting
import time

store = 1
print('Choose your Store:')
print('1 for Amazon\n2 for Costco\n3 for North Face\n4 for Kmart\n5 for Home Depot')
store = input()
minSupt = float(input('Enter minimum support (in %)'))
minConf = float(input('Enter minimum confidence (in %)'))
fileObject = open('./StoresDB/store'+str(store)+'.txt', 'r')

transactions = []
Singleitems = []
OccurenceCountTemp = {}
OccurenceCount = {}
HashCode = {}
TotalTransact = 0
OccurenceCountBF = {}

start_time = time.time()

#Scan all the trasactions to generate list of SingleItems
for row in fileObject:
    tokenList = row.split(' ')
    ThisTrans = tokenList[1].rstrip('\n').split(',')
    transactions.append(ThisTrans)
    TotalTransact += 1 
    for item in ThisTrans: 
        if item not in Singleitems:
            Singleitems.append(item)
            OccurenceCountTemp[item] = 1
        else:
            OccurenceCountTemp[item] += 1
Singleitems.sort()

FST1 = []
FST = []
NFST_hash = {}
NFST = []
k = 2
minoccur = (minSupt * TotalTransact )/100
minConf = minConf/100

for item in Singleitems :
    index = Singleitems.index(item)
    HashCode[item] = index
    if OccurenceCountTemp[item] >= minoccur :
        OccurenceCount[HashCode[item]] = OccurenceCountTemp[item]
        OccurenceCountBF[HashCode[item]] = OccurenceCountTemp[item]
        temp = [item]
        FST1.append(temp)
this_FST = []
prev_FST = []
FST = []
k = 2
time_pause_Bf_start = time.time()

prev_FST = FST1
FST = FST1

#Generate all FSTs by Apriori Alg
while True :
    this_FST = getFrequentItemSets_Apr(prev_FST,k,minoccur)
    FST += this_FST
    prev_FST = this_FST
    k += 1
    if len(this_FST) < 2 :
        break

asrAP = genAssrules(FST)
end_time_apr = time.time()
time_pause_Bf_stop = time.time()

#Generate all FSTs by Brute Force Alg
FST_bf = []
n = len(Singleitems)
for CombLent in range(1,n):
    CombinationList = []
    this_FST_bf = []
    temp_list = [None]*CombLent
    GenCombin(Singleitems,temp_list,0,n-1,0,CombLent)
    this_FST_bf = getFSTfromAllComb(CombinationList)
    FST_bf += this_FST_bf

asrBF = genAssrules(FST_bf,2)
end_time_bf = time.time()

apr_time = round((end_time_apr - start_time) *1000 ,2)
bf_time = round((end_time_bf - start_time - (time_pause_Bf_stop - time_pause_Bf_start))*1000,2)
time_impr = bf_time - apr_time
print("\n\n\nTransactions\n\n")
for trans in transactions:
    print(trans,"\n")

print("\nUser Chose MinSupport :",minSupt," and MinConfidence: ",minConf*100)
print("\nAssociation rules generated by Apriori Algorithm: ")
for elem in asrAP:
    print("\n",elem[0],"----->",elem[1],"   Sup: ",elem[2],"  Conf: ",elem[3])
print("\n\nAssociation rules generated by Brute Force Algorithm: ")
for elem in asrBF:
    print("\n",elem[0],"----->",elem[1],"   Sup: ",elem[2],"  Conf: ",elem[3])
print("\nTotal Association rules generated by Apriori are : ",len(asrAP)," and by Brute Force are: ",len(asrBF))    
print("\nTime taken for generating Association rules from Apriori is :",apr_time,"ms and from Brute Force is :",bf_time,"ms" )
print("Time improvement by Apriori Alg : ",time_impr,"ms")






Choose your Store:
1 for Amazon
2 for Costco
3 for North Face
4 for Kmart
5 for Home Depot
1
Enter minimum support (in %)10
Enter minimum confidence (in %)10



Transactions


['PHONE', 'PHONE_CASE', 'PHONE_CHARGER'] 

['PHONE', 'HEADPHONE', 'SCREEN_PROTECTOR'] 

['LAPTOP', 'LAPTOP_BAG', 'WIRELESS_MOUSE', 'LAPTOP_DESK'] 

['HARD_DISK', 'PHONE_CHARGER', 'HEADPHONE'] 

['PHONE', 'PHONE_CASE', 'SCREEN_PROTECTOR'] 

['LAPTOP', 'HARD_DISK'] 

['LAPTOP_DESK', 'WIRELESS_MOUSE', 'PHONE_CHARGER'] 

['PHONE', 'PHONE_CASE'] 

['LAPTOP', 'LAPTOP_BAG'] 

['PHONE_CASE', 'SCREEN_PROTECTOR'] 

['PHONE', 'HEADPHONE', 'SCREEN_PROTECTOR'] 

['LAPTOP', 'WIRELESS_MOUSE'] 

['LAPTOP', 'PHONE', 'PHONE_CHARGER'] 

['PHONE', 'LAPTOP'] 

['HARD_DISK', 'HEADPHONE'] 

['LAPTOP', 'SCREEN_PROTECTOR'] 

['PHONE', 'LAPTOP', 'LAPTOP_BAG'] 

['PHONE', 'HEADPHONE'] 

['PHONE', 'HARD_DISK'] 

['LAPTOP_BAG', 'WIRELESS_MOUSE'] 


User Chose MinSupport : 10.0  and MinConfidence:  10.0

Association rules generated by Apriori

In [ ]:

len(Singleitems)

10

In [ ]:
Singleitems

['HARD_DISK',
 'HEADPHONE',
 'LAPTOP',
 'LAPTOP_BAG',
 'LAPTOP_DESK',
 'PHONE',
 'PHONE_CASE',
 'PHONE_CHARGER',
 'SCREEN_PROTECTOR',
 'WIRELESS_MOUSE']